# Project Title: The Impact of News on the Market
### •	Team Members:
##### 	Rachel Torres, Christian Attard, Jess Alcalde, Nitin Khade
### •	Project Description/Outline:
##### -	We will look at news data and stock data to determine the effects of the news on how the market behaves.
### •	Research Questions to Answer:
##### -	How do news headlines affect the stock market?
##### -	Is there any correlation between certain types of headlines and effects on the market?
##### -	Does negative news affect stocks greater than positive or neutral?
##### -	Can we assign a factor(weighting) to it?

In [19]:
# import dependencies
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

from news_api import api_key
from x_api import x_api_key

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

# pip install vaderSentiment


### We'll query the news api to gather news headlines from the web

In [20]:
# parameters for news api
q='politics',
from_param='2019-05-24',
language='en',
sort_by='relevancy',
page = 4,
pageSize = 50

url = f'https://newsapi.org/v2/everything?q={q}&from={from_param}&language={language}&sortBy={sort_by}&pageSize={pageSize}&page={page}&apiKey={api_key}'
response = requests.get(url)

url_2 = f'https://newsapi.org/v2/everything?q=apple&from=2019-05-24&to=2019-06-23&language=en&sortBy=popularity&pageSize={pageSize}&page=3&apiKey={api_key}'
response = requests.get(url_2)
# Generate response into json 

In [21]:
pprint(response.json())

{'code': 'parameterInvalid',
 'message': 'You are trying to request results too far in the past. Your plan '
            'permits you to request articles as far back as 2019-05-25, but '
            'you have requested 2019-05-24. To extend this please upgrade to a '
            'paid plan.',
 'status': 'error'}


### We'll use a different API to gather news data since we have a restriction on the news api

In [22]:
# Query contextual web search API
# Replace the following string value with your valid X-RapidAPI-Key.
Your_X_RapidAPI_Key = x_api_key;

# The query parameters: (update according to your search query)
q = "Donald%20Trump%20News" #the search query
pageNumber = 1 #the number of requested page
pageSize = 50 #the size of a page
autoCorrect = True #autoCorrectspelling
safeSearch = False #filter results for adult content


# test response to query and get count of total items and pages
response_test=requests.get("https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/Search/NewsSearchAPI?q={}&pageNumber={}&pageSize={}&autocorrect={}&safeSearch={}".format(q, pageNumber, pageSize, autoCorrect,safeSearch),
headers={
"X-RapidAPI-Key": Your_X_RapidAPI_Key
}
).json()

#Get the number of items returned
totalCount = response_test["totalCount"];
totalPages = round(totalCount/pageSize)


In [23]:
pprint(response_test)

{'_type': 'news',
 'didUMean': '',
 'relatedSearch': ['president',
                   'iran',
                   'washington',
                   'fox',
                   'white house',
                   'united states',
                   'views',
                   'abc',
                   'says rape accuser totally',
                   'president <b>donald</b> j',
                   'took in as much',
                   'off strike on iran'],
 'totalCount': 2475,
 'value': [{'datePublished': '2019-06-18T07:10:00',
            'description': 'Get breaking national and world <b>news</b>, '
                           'broadcast video coverage, and exclusive '
                           'interviews. Find the top <b>news</b> online at ABC '
                           '<b>news</b>.',
            'image': {'base64Encoding': None,
                      'height': 558,
                      'thumbnail': 'https://contextualwebsearch.com/api/thumbnail/get?value=3712910521017174876',
        

            'url': 'https://www.pmnewsnigeria.com/2019/06/23/us-sanctions-biting-iran-bolton/'},
           {'datePublished': '2019-06-23T10:11:10',
            'description': 'President <b>Donald Trump</b> sent North Korean '
                           'leader Kim Jong Un an "excellent" letter, the '
                           "North's state-run <b>news</b> agency reported "
                           'Sunday, quoting Kim as saying he would "seriously '
                           'contemplate it."',
            'image': {'base64Encoding': None,
                      'height': 1368,
                      'thumbnail': 'https://contextualwebsearch.com/api/thumbnail/get?value=5214114028529950531',
                      'thumbnailHeight': 192,
                      'thumbnailWidth': 287,
                      'url': 'https://twt-thumbs.washtimes.com/media/image/2019/04/23/putin_kim_summit_sanctions_38547_s2048x1368.jpg?fd42bbe046ffd948e034ff7a7a81093fe3f86903',
                      'width

In [24]:
# generate sample response to look at headers
pprint((response_test['value'][2]))

{'datePublished': '2019-06-26T09:28:54',
 'description': 'Work continued on a possible meeting between Russian '
                'President Vladimir Putin and U.S. President <b>Donald '
                'Trump</b> on the sidelines of a G20 meeting this week, a '
                'Kremlin spokesman told reporters on Wednesday.  World  One '
                '<b>News</b> Page: Wednesday, 26 June 2019',
 'image': {'base64Encoding': None,
           'height': 256,
           'thumbnail': 'https://contextualwebsearch.com/api/thumbnail/get?value=6187294181911490305',
           'thumbnailHeight': 256,
           'thumbnailWidth': 256,
           'url': 'https://cdn.newsserve.net/ONPglobe256.png',
           'width': 256},
 'isSafe': True,
 'keywords': 'donald trump,news',
 'language': 'en',
 'provider': {'name': 'onenewspage'},
 'title': 'Kremlin says work continues on possible Putin-Trump meeting',
 'url': 'https://www.onenewspage.com/n/World/1zkidworiz/Kremlin-says-work-continues-on-possible-

In [25]:
print(f' There are {totalPages} pages, with {totalCount} total articles.')

 There are 50 pages, with 2475 total articles.


In [26]:
# create empty lists to hold variable results
url_list = []
title_list = []
description_list = []
keywords_list = []
provider_list = []
date_list = []

# Load news data into pandas dataframe
# page = 1
for page in range(1,totalPages+1):
    response=requests.get("https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/Search/NewsSearchAPI?q={}&pageNumber={}&pageSize={}&autocorrect={}&safeSearch={}".format(q, page, pageSize, autoCorrect,safeSearch),
headers={"X-RapidAPI-Key": Your_X_RapidAPI_Key}).json()

# print(response)
    try:
        #Go over each resulting item
        for webPage in response["value"]:

#Get the web page metadata
            url = webPage["url"]
            title = webPage["title"]
            description = webPage["description"]
            keywords = webPage["keywords"]
            provider = webPage["provider"]["name"]
            datePublished = webPage["datePublished"]
        
        
            url_list.append(url)
            title_list.append(title)
            description_list.append(description)
            keywords_list.append(keywords)
            provider_list.append(provider)
            date_list.append(datePublished)

            print(f'printing {title}')    
    except (KeyError,IndexError):
            print('Not found, skipping')


printing <b>Trump</b> threatens to deport millions beginning next week
printing Election 2020: Democratic debate preview
printing Kremlin says work continues on possible Putin-Trump meeting
printing LIVE: Double blow to president as Mueller agrees to testify on Russia evidence and judge opens up his financial records
printing Impersonators of <b>Donald Trump</b> and Kim Jong-un tours Osaka ahead of G-20 Summit
printing Foxs Sean Hannity,Tucker Carlson enjoy <b>Trump</b> rally ratings bonanza
printing Foxs Hannity, Carlson Enjoy <b>Trump</b> Rally Ratings Bonanza
printing Glendale man charged in theft of Marilyn Monroe statue pleads not guilty  Pasadena Star <b>News</b>
printing What you need to know about the US-Iran crisis, United States <b>News</b> & Top Stories
printing Fox's Hannity, Carlson enjoy <b>Trump</b> rally ratings bonanza
printing US-Iran crisis: <b>Trump</b> lashes out at 'ignorant and insulting' statement
printing Why Palestine needs help
printing Joy Reid oddly accuses

printing Asian shares mixed as markets await <b>news</b> on China-US trade
printing Occupy World Writes
printing <b>Donald Trump's</b> Role in the Central Park 5 Case, Based on 'When They See Us' Netflix Doc
printing San Francisco shooting suspect arrested in Oakland  The Mercury <b>News</b>
printing Fast <b>News</b>: Todays top stories
printing AP FACT CHECK: <b>Trump</b> and a tale of 2 sheets of paper :: WRAL.com
printing <b>Donald Trump</b> Protest Grows at <b>Trump</b> Tower
printing After Extended Delay, Senate Approves Disaster Relief Funds For Puerto Rico
printing <b>Trump</b> tries to steer border wall deal to North Dakota firm
printing NY legislation could release <b>Trump</b> tax returns, allows state prosecution despite federal pardon
printing Blame them, says Chinese ambassador on sinking US-China trade deal
printing All the Times John Bolton Called for War with Iran
printing Fox <b>News</b> Host Pete Hegseth Privately Lobbied <b>Trump</b> to Pardon Accused War Criminals
p

printing Writer accuses <b>Trump</b> of sexual assault more than two decades ago, an allegation he denies, United States <b>News</b> & Top Stories
printing Why <b>Trump</b> called off Iran attack: He listened to the sceptics, United States <b>News</b> & Top Stories
printing US CENTCOM releases video of Iran shooting down its RQ-4A Global Hawk drone
printing A half-hour away: How <b>Trump</b> opted against Iran strike
printing US warns India of more action against unfair trade practices
printing <b>Trump</b> nominates Mark Esper as Secretary of Defense: White House
printing <b>News</b> in 90 Seconds
printing <b>Trump</b> nominates Mark Esper as Secretary of Defence: White House, United States <b>News</b> & Top Stories
printing Threats of deportation raids spark fear in Utah immigrant community
printing Cocked and loaded: <b>Trump</b> blinks, Iran gloats
printing Utah Sens. Mike Lee and Mitt Romney, Rep. Chris Stewart say <b>Trump</b> made right call in backing off military strike on Ira

printing <b>Trump news</b>  live: 'First-hand accounts' of president's obstruction to be revealed after Justice Department deal over Mueller evidence
printing <b>Trump</b> ready to slap more tariffs on China after G20 meeting
printing Asian shares rise as <b>Trump</b> plans to meet Chinese leader
printing Mexico denies <b>Trump's</b> claim of secret concessions in deal
printing House Democrats make deal to see Mueller files on <b>Trump</b>
printing Mexico Denies <b>Trumps</b> Claim Of Secret Concessions In Deal
printing for now, it's Democratic frontrunner Biden, United States <b>News</b> & Top Stories
printing US House Democrats to get more Mueller evidence, <b>Trump</b> calls witness 'sleazebag', United States <b>News</b> & Top Stories
printing XE Morning Update
printing <b>Trump</b> says United Tech, Raytheon deal may hurt competition, United States <b>News</b> & Top Stories
printing With annexation comment, Friedman just made Netanyahus life more difficult
printing <b>Trump</b> ins

printing <b>Donald Trump</b> Arrives in United Kingdom for State Visit (VIDEO)
printing Local <b>news</b> from regional <b>news</b>papers and blogs
printing Edinburgh Dungeon 'bans <b>Donald Trump</b> supporters' during state visit
printing <b>Trump</b> wouldnt be president if not for Twitter: Sacha Baron Cohen
printing US President <b>Trump</b> urges UK to be 'very careful' over Huawei and 5G ahead of visit, Europe <b>News</b> & Top Stories
printing <b>Donald Trump</b>: send in Nigel Farage to negotiate with the EU
printing <b>Trump</b> backs Boris as next British premier, Europe <b>News</b> & Top Stories
printing <b>Trump</b> overruled Kushner and other advisers in punishing Mexico, United States <b>News</b> & Top Stories
printing Hope in 'dark age'
printing It's not just Mexico. <b>Trump</b> is taking on free trade dogma, United States <b>News</b> & Top Stories
printing <b>Trump</b> says Boris Johnson would be 'very good' as next British prime minister, Europe <b>News</b> & Top Stor

...

printing At the border, <b>Trump</b> moves closer to emergency declaration
printing Government Shutdown Stymies Immigrants Asylum Cases
printing <b>Trump</b> to honor champion Clemson  at White House on Monday
printing Tammy Duckworth reacts to Trump’s national emergency threat
printing GOP rejected Obama’s executive reach, but accepts Trump’s
printing SHUTDOWN DAY 20: <b>Trump</b> Visists Texas Border, Threatens to Declare National Emergency to Build The Wall [VIDEOS]
printing <b>Trump</b> cancels Davos trip over US-Mexico border wall row
printing SHUTDOWN DAY 19: <b>Trump</b> Walks Out of Meeting With Democrat Leaders: BYE BYE
printing Kim, <b>Trump</b> to meet again
printing <b>Trump</b> storms out of talks
printing <b>Trump</b> demands $5.7 bn for steel barrier on US-Mexico border in Oval Office speech
printing Schedule Change Jan. 8
printing <b>Trump</b> ramps up Mexico wall row with speech, border visit » Capital <b>News</b>
printing Democrats ramp up pressure
printing <b>Trumps<

printing Sarah Sanders can't name a single 'fake <b>news</b>' outlet
printing Fox <b>News</b> Shepard Smith debunks <b>Trumps</b> migrant caravan rhetoric: There is no invasion
printing <b>Trump's</b> phony 'caravan' hysteria led to the Synagogue murders: Adam Serwer for The Atlantic
printing <b>Trump</b> Says Pittsburgh Shooting Has Little to Do With Gun Laws
printing <b>Trump</b> hints he could curb immigration by decree » Capital <b>News</b>
printing Chilling mail-bomb plot
printing Khashoggi fiancée snubs <b>Trump</b> invitation
printing Seeking 'Unprecedented' Pressure on Iran, <b>Trump</b> Reimposes US Sanctions Lifted After Nuclear Deal
printing 5 things to know about <b>Trump's</b> war (of words) on the Fed
printing Bombs sent to <b>Trump</b> critics Biden, De Niro as manhunt intensifies
printing US probes pipe bombs sent to <b>Trump</b> foes Obama, Clinton, CNN
printing <b>Trump</b> Says Saudis Staged 'Worst Cover-Up Ever' on Khashoggi
printing 2018 election: No, this new Hous

printing <b>Trump</b> mulls changes
printing Dismantling nuclear arsenal
printing <b>Trump</b> bashes 'witch hunt'
printing 'Kinky <b>Trump</b>' doughnut
printing White House invites Putin
printing <b>Trump</b> at odds with chiefs
printing Historians on <b>Trump</b>
printing <b>Trump</b> on defensive
printing Obama talking to <b>Trump</b>?
printing China challenges tariffs
printing <b>Trump</b>: queen 'beautiful'
printing Explosive <b>Trump</b> interview
printing <b>Trump's</b> claims of success
printing NATO insists it's united
printing <b>Trump</b> met by protest, pomp
printing <b>Trump</b> to meet Queen
printing <b>Trump</b> out of picks
printing NATO boosts spending
printing Estefans: Immigration debate has fueled reaction to musical – WSVN 7News
printing <b>Trump</b> on offensive
printing Defence by the numbers
printing <b>Trump</b> pardons ranchers
printing <b>Trump</b> picks Kavanaugh
printing China and Russia hit back at <b>Trump</b> tariffs
printing <b>Trump</b> to meet May, Q

printing <b>Trump</b> warns Iran
printing Senate Committee Paves Way for Pompeo to Become Top US Diplomat
printing Trump’s divisive pick to run NASA wins narrow confirmation – WSVN 7News
printing <b>Trump</b> to further 'good relationship' with Kim Jong-un
printing <b>Trump</b>, Abe to meet despite strain over North Korea, tariffs – WSVN 7News
printing <b>Trump</b> suspends metal tariffs for key US trade partners, United States <b>News</b> & Top Stories
printing ‘Godmother of Torture’ Haspel should be in dock at The Hague, not head of CIA – John Kiriakou — RT USA <b>News</b>
printing <b>Trump</b> praises House GOP report of no collusion on Russia – WSVN 7News
printing Granlund cartoon: North Korea summit
printing Beeler cartoon: Trade war
printing AP-NORC Poll: Nearly 6 in 10 Americans say <b>Trump</b> a racist – WSVN 7News
printing <b>Trump</b> urges ban on gun devices like bump stocks – WSVN 7News
printing ABC Hypes Claims of 'Racist' Ingraham Comments, Edits Her Response
printing Pr

printing Governor: Dumping migrants in Florida is unacceptable
printing Weatherford cartoon: Another crisis
printing '<b>Donald Trump</b> is the strongest supporter of Israel since Reagan'
printing PM's son blasts New York Times
printing College hoops champs Virginia won't be visiting White House
printing Iowa GOP lawmakers approve limit to attorney general powers
printing This 2020 presidential campaign scenario seems familiar
printing Sanders Internal Poll: Bernie Beats <b>Trump</b> in Wisconsin, Michigan, Pennsylvania
printing US Congress approves Colorado River drought plan
printing CERAWeek Diary: Pompeo pushes CV with hymn of praise to America’s ‘new age of discovery’
printing Schiff: Pelosi ‘absolutely right’ to hold back impeachment
printing Washington Gov. Inslee joins Democratic presidential field
printing Report: North Korea Continues Christian Crackdown
printing Near Kim hotel, North Korean defectors wish for peace
printing TV footage shows Kim Jong Un taking smoke break
pr

In [27]:
# assign list to dataframe

newsFrame = pd.DataFrame({'datePublished':date_list,'description':description_list,'keywords':keywords_list,'provider':provider_list,'title':title_list,'url':url_list})


In [28]:
newsFrame.head(200)

,datePublished,description,keywords,provider,title,url
0,2019-06-18T07:10:00,"Get breaking national and world <b>news</b>, b...","donald trump news,abc news,week",go,<b>Trump</b> threatens to deport millions begi...,https://abcnews.go.com/Politics/wireStory/trum...
1,2019-06-26T12:00:00,With the Illinois primary still another 265 da...,"donald trump,democrats,miami,news",news-gazette,Election 2020: Democratic debate preview,http://www.news-gazette.com/news/local/2019-06...
2,2019-06-26T09:28:54,Work continued on a possible meeting between R...,"donald trump,news",onenewspage,Kremlin says work continues on possible Putin-...,https://www.onenewspage.com/n/World/1zkidworiz...
3,2019-06-26T07:07:00,<b>Donald</b> Trumphas received two fresh setb...,"donald trump,trump news,president,house",independent,LIVE: Double blow to president as Mueller agre...,https://www.independent.co.uk/news/world/ameri...
4,2019-06-26T07:03:34,"The impersonators of rivals, North Korean lead...","president donald trump,news news,kim",indianexpress,Impersonators of <b>Donald Trump</b> and Kim J...,https://indianexpress.com/photos/world-news/im...
5,2019-06-26T03:50:00,LOS ANGELES >> Ratings figures show Fox <b>New...,"president donald trump,fox news",staradvertiser,"Foxs Sean Hannity,Tucker Carlson enjoy <b>Trum...",https://www.staradvertiser.com/2019/06/25/news...
6,2019-06-26T02:00:32,Ratings figures show Fox <b>News</b> hosts Sea...,"president donald trump,news",theyeshivaworld,"Foxs Hannity, Carlson Enjoy <b>Trump</b> Rally...",https://www.theyeshivaworld.com/news/general/1...
7,2019-06-26T02:00:07,"Suspect Austin Mikel Clay, 25, is the same man...","president donald trump,news,man",pasadenastarnews,Glendale man charged in theft of Marilyn Monro...,https://www.pasadenastarnews.com/2019/06/25/gl...
8,2019-06-26T00:53:13,WASHINGTON - US President <b>Donald Trump</b> ...,"president donald trump,iran,news",straitstimes,What you need to know about the US-Iran crisis...,http://www.straitstimes.com/world/united-state...
9,2019-06-25T23:32:09,LOS ANGELES Fox <b>News</b> hosts Sean Hannit...,"president donald trump,fox news",660citynews,"Fox's Hannity, Carlson enjoy <b>Trump</b> rall...",https://www.660citynews.com/2019/06/25/foxs-ha...


In [29]:
newsFrame.dtypes

datePublished    object
description      object
keywords         object
provider         object
title            object
url              object
dtype: object

In [32]:
# clean description and title columns
newsFrame.description = newsFrame.description.str.replace('<b>','')
newsFrame.description = newsFrame.description.str.replace('</b>','')
newsFrame.title = newsFrame.title.str.replace('<b>','')
newsFrame.title = newsFrame.title.str.replace('</b>','')

In [33]:
newsFrame.head()

,datePublished,description,keywords,provider,title,url
0,2019-06-18T07:10:00,"Get breaking national and world news, broadcas...","donald trump news,abc news,week",go,Trump threatens to deport millions beginning n...,https://abcnews.go.com/Politics/wireStory/trum...
1,2019-06-26T12:00:00,With the Illinois primary still another 265 da...,"donald trump,democrats,miami,news",news-gazette,Election 2020: Democratic debate preview,http://www.news-gazette.com/news/local/2019-06...
2,2019-06-26T09:28:54,Work continued on a possible meeting between R...,"donald trump,news",onenewspage,Kremlin says work continues on possible Putin-...,https://www.onenewspage.com/n/World/1zkidworiz...
3,2019-06-26T07:07:00,Donald Trumphas received two fresh setbacksas ...,"donald trump,trump news,president,house",independent,LIVE: Double blow to president as Mueller agre...,https://www.independent.co.uk/news/world/ameri...
4,2019-06-26T07:03:34,"The impersonators of rivals, North Korean lead...","president donald trump,news news,kim",indianexpress,Impersonators of Donald Trump and Kim Jong-un ...,https://indianexpress.com/photos/world-news/im...


In [34]:
newsFrame.count()

datePublished    2032
description      2032
keywords         2032
provider         2032
title            2032
url              2032
dtype: int64

In [35]:
# Query sample news description
newsFrame.iloc[7][1]

'Suspect Austin Mikel Clay, 25, is the same man who used a pickax to vandalize President Donald Trumps Walk of Fame star.'

In [ ]:
# import stock data 

# Load stock data into notebook as dataframe

In [ ]:
# Merge 2 DataFrames

In [ ]:
# Do sentiment analysis of news data

In [ ]:
# Use Matplotlib and stats to generate graphs and look for relationships